# Jupyter notebook based on ImageD11 to process scanning 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 26/02/2024

## NOTE: These notebooks are under active development
They require the latest version of ImageD11 from Git to run.

If you don't have this set up yet, you can run the below cell.

It will automatically download and install ImageD11 to your home directory

In [ ]:
import os

username = os.environ.get("USER")

# USER: You can change this location if you want

id11_code_path = f"/home/esrf/{username}/Code/ImageD11"

# check whether we already have ImageD11 here

if os.path.exists(id11_code_path):
    raise FileExistsError("ImageD11 already present! Giving up")

!git clone https://github.com/FABLE-3DXRD/ImageD11 {id11_code_path}
output = !cd {id11_code_path} && python setup.py build_ext --inplace

if not os.path.exists(os.path.join(id11_code_path, "build")):
    raise FileNotFoundError(f"Can't find build folder in {id11_code_path}, compilation went wrong somewhere")

import sys

sys.path.insert(0, id11_code_path)

# if this works, we installed ImageD11 properly!
try:
    import ImageD11.cImageD11
except:
    raise FileNotFoundError("Couldn't import cImageD11, there's a problem with your Git install!")

In [ ]:
import os

username = os.environ.get("USER")

# USER: Change the path below to point to your local copy of ImageD11:

id11_code_path = f"/home/esrf/{username}/Code/ImageD11"

if not os.path.exists(id11_code_path):
    raise FileNotFoundError("Can't find Git version of ImageD11 in {id11_code_path}")

import sys

sys.path.insert(0, id11_code_path)

try:
    import ImageD11.cImageD11
except:
    raise FileNotFoundError("Couldn't import cImageD11, there's a problem with your Git install!")

In [ ]:
# import functions we need

import glob, pprint
import fabio
import time
import shutil

import ImageD11.sinograms.dataset
import ImageD11.sinograms.lima_segmenter
import ImageD11.sinograms.assemble_label
import ImageD11.sinograms.properties
import ImageD11.nbGui.nb_utils as utils

import numpy as np
import fabio
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from skimage import filters, measure, morphology
from ipywidgets import interact, interactive, widgets, fixed, Layout
import h5py

%matplotlib ipympl

In [ ]:
# Check that we're importing ImageD11 from the home directory rather than from the Jupyter kernel

?ImageD11.sinograms.dataset

In [ ]:
# NOTE: For old datasets before the new directory layout structure, we don't distinguish between RAW_DATA and PROCESSED_DATA

### USER: specify your experimental directory

rawdata_path = "/home/esrf/james1997a/Data/ihma439/id11/20231211/RAW_DATA"

!ls -lrt {rawdata_path}

### USER: specify where you want your processed data to go

processed_data_root_dir = "/home/esrf/james1997a/Data/ihma439/id11/20231211/PROCESSED_DATA/James/20240226"

In [ ]:
# USER: pick a sample and a dataset you want to segment

sample = "FeAu_0p5_tR_nscope"
dataset = "top_100um"

# USER: specify path to detector mask

# mask_path = '/data/id11/inhouse1/ewoks/detectors/files/eiger_E-08-0173/mask_with_gaps_E-08-0173.edf'  # temporary eiger mask (Nov 2023)
# mask_path = '/data/id11/inhouse1/ewoks/detectors/files/eiger_E-08-0144/mask.edf'  # normal eiger mask

mask_path = '/data/id11/inhouse1/ewoks/detectors/files/eiger_E-08-0173/mask_with_gaps_E-08-0173.edf'  # temporary eiger mask (Nov 2023)

In [ ]:
# create ImageD11 dataset object

ds = ImageD11.sinograms.dataset.DataSet(dataroot=rawdata_path,
                                        analysisroot=processed_data_root_dir,
                                        sample=sample,
                                        dset=dataset)
ds.import_all()
ds.save()

In [ ]:
# TODO: autodetect eiger/frelon (get from ds object)

In [ ]:
#Define the initial parameters
start_pars = {#"bgfile": bg_path,
              "maskfile": mask_path,
              "cut": 1,
              "pixels_in_spot": 3}

mask = fabio.open(start_pars["maskfile"]).data
# convert to boolean
mask = mask.astype(bool)

# check if mask is mostly zeros or mostly ones
mask_ones_fraction = np.sum(mask.ravel())/mask.ravel().shape[0]
if mask_ones_fraction > 0.5:
    print("Inverting mask!")
    # mask is mostly ones, we should invert it
    mask = ~mask

# import the image
with h5py.File(ds.masterfile, 'r') as h5In:
    # get middle scan
    sorted_keys = sorted(h5In.keys(), key=lambda x: float(x))
    middle_scan = sorted_keys[len(sorted_keys)//2]
    single_scan_shape = h5In[f'{middle_scan}/measurement/eiger'].shape[0]
    detector_frame = h5In[f'{middle_scan}/measurement/eiger'][single_scan_shape//2].astype('uint16')

#bgimage = fabio.open(start_pars["bgfile"]).data
# no background
bgimage = np.zeros_like(detector_frame)

def segment_image(image, cut, pixels_in_spot, bgimage):
    bgsub = image - bgimage
    image_masked = image * ~mask
    bgsub_masked = bgsub * ~mask
    cut_image = bgsub_masked > cut
    labeled_image = measure.label(cut_image)
    regions = measure.regionprops(labeled_image)
    blob_mask = np.zeros_like(image, dtype=bool)
    spot_count = 0
    for region in regions:
        if region.area >= pixels_in_spot:
            blob_mask[labeled_image == region.label] = 1
            spot_count += 1
    filtered_image = image_masked * blob_mask
    return filtered_image, spot_count


cut_slider = widgets.IntSlider(value=start_pars["cut"], min=1, max=20, step=1, description='Cut:')
pixels_in_spot_slider = widgets.IntSlider(value=start_pars["pixels_in_spot"], min=1, max=20, step=1, description='Pixels in Spot:')

# Display the image initially
plt.figure()

filtered_image, nspots = segment_image(detector_frame, cut=cut_slider.value, pixels_in_spot=pixels_in_spot_slider.value, bgimage=bgimage)
im = plt.imshow(filtered_image, cmap="viridis", norm=LogNorm(vmin=1, vmax=1000), interpolation="nearest")
plt.title(f"Scan {middle_scan}, frame {single_scan_shape//2}\n cut={cut_slider.value}, pixels_in_spot={pixels_in_spot_slider.value}, nspots={nspots}")
plt.show()

def update_image(cut, pixels_in_spot):
    filtered_image, nspots = segment_image(detector_frame, cut, pixels_in_spot, bgimage)
    im.set_data(filtered_image)
    plt.title(f"Scan {middle_scan}, frame {single_scan_shape//2}\n cut={cut}, pixels_in_spot={pixels_in_spot}, nspots={nspots}")
    plt.draw()

interactive_plot = widgets.interactive(update_image, cut=cut_slider, pixels_in_spot=pixels_in_spot_slider)

display(interactive_plot)

In [ ]:
end_pars = {# "bgfile": bg_path,
              "maskfile": mask_path,
              "cut": cut_slider.value,
              "pixels_in_spot": pixels_in_spot_slider.value}

In [ ]:
# create batch file to send to SLURM cluster

sbat = ImageD11.sinograms.lima_segmenter.setup(ds.dsfile, **end_pars)
if sbat is None:
    raise ValueError("This scan has already been segmented!")
print(sbat)

In [ ]:
utils.slurm_submit_and_wait(sbat, 60)

In [ ]:
# label sparse peaks

ImageD11.sinograms.assemble_label.main(ds.dsfile, ds.sparsefile)

In [ ]:
# generate peaks table

ImageD11.sinograms.properties.main(ds.dsfile, ds.sparsefile, ds.pksfile, options={'algorithm': 'lmlabel', 'wtmax': 70000, 'save_overlaps': False})

In [ ]:
# make a new subfolder called "sparse" that holds all the individual "scan______sparse.h5" files

sparse_folder_path = os.path.join(ds.analysispath, "sparse")

if not os.path.exists(sparse_folder_path):
    os.mkdir(sparse_folder_path)
    
scan_sparse_files = glob.glob(os.path.join(ds.analysispath, "scan*_sparse.h5"))

for scan_sparse_file in scan_sparse_files:
    shutil.move(scan_sparse_file, sparse_folder_path)

In [ ]:
# TODO: incorporate DATA/visitor/ma5839/id11/20240118/SCRIPTS/0_S3DXRD_segment_and_label_single_dset.ipynb

In [ ]:
if 1:
    raise ValueError("Change the 1 above to 0 to allow 'Run all cells' in the notebook")

In [ ]:
# Now that we're happy with our segmentation parameters, we can run the below cell to do this in bulk for many samples/datasets
# by default this will do all samples in sample_list, all datasets with a prefix of dset_prefix
# you can add samples and datasets to skip

skips_dict = {
    "FeAu_0p5_tR_nscope": ["top_-50um", "top_-100um"]
}

dset_prefix = "top"

sample_list = ["FeAu_0p5_tR_nscope"]
    
samples_dict = utils.find_datasets_to_process(rawdata_path, skips_dict, dset_prefix, sample_list)
    
# manual override:
# samples_dict = {"FeAu_0p5_tR_nscope": ["top_100um", "top_200um"]}
    
# now we have our samples_dict, we can process our data:
mask_path = '/data/id11/nanoscope/Eiger/eiger_mask_E-08-0173_20231127.edf'

# you can change these if needed, but they will default to those you selected with the widget
seg_pars = {"maskfile": mask_path,
            "cut": cut_slider.value,
            "pixels_in_spot": pixels_in_spot_slider.value}

for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
        
        ds = ImageD11.sinograms.dataset.DataSet(dataroot=rawdata_path,
                                                analysisroot=processed_data_root_dir,
                                                sample=sample,
                                                dset=dataset)
        if os.path.exists(ds.sparsefile):
            print(f"Found existing Sparse file for {dataset} in sample {sample}, skipping")
            continue
        
        print("Importing DataSet object")
        try:
            ds.import_all()
        except ValueError:
            print(f"Very dodgy scan! Skipping")
            continue
        except KeyError:
            print(f"Very dodgy scan! Skipping")
            continue
        print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
        ds.save()
        
        print("Segmenting")
        sbat = ImageD11.sinograms.lima_segmenter.setup(ds.dsfile, **seg_pars)
        
        if sbat is None:
            print(f"{dataset} in sample {sample} already segmented, skipping")
            continue
        
        utils.slurm_submit_and_wait(sbat, 60)
        
        print("Labelling sparse peaks")
        ImageD11.sinograms.assemble_label.main(ds.dsfile, ds.sparsefile)
        
        print("Generating peaks table")
        ImageD11.sinograms.properties.main(ds.dsfile, ds.sparsefile, ds.pksfile, options={'algorithm': 'lmlabel', 'wtmax': 70000, 'save_overlaps': False})
        
        print("Cleaning up sparse files")
        sparse_folder_path = os.path.join(ds.analysispath, "sparse")

        if not os.path.exists(sparse_folder_path):
            os.mkdir(sparse_folder_path)

        scan_sparse_files = glob.glob(os.path.join(ds.analysispath, "scan*_sparse.h5"))

        for scan_sparse_file in scan_sparse_files:
            shutil.move(scan_sparse_file, sparse_folder_path)
print("Done!")